### Set up environment

In [ ]:
# efficientnet-b0-224
# efficientnet-b1-240
# efficientnet-b2-260
# efficientnet-b3-300
# efficientnet-b4-380
# efficientnet-b5-456
# efficientnet-b6-528
# efficientnet-b7-600

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

from google.colab import drive
drive.mount('/content/drive')

!ln -s /content/drive/My\ Drive/Colab\ Notebooks/hku-oph/* /content/
%cd /content/src

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
        
# ML libraries required
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.metrics import KappaScore # solution evaluated with qudratic kappa
from fastai.tabular import * # for ensemble model training
import torch
# efficientnet is not integrated into fastai yet


# Other libraries required
import matplotlib.pyplot as plt
from models.efficientnet_pytorch import EfficientNet

# garbage collector
import gc

import random
from datetime import datetime

def seed_everything(seed_value, use_cuda=True):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
    
seed_everything(42, True)

### Define model details

In [ ]:
import json
models_config = {}
only_train_model = None
with open('models_config.json') as f:
    models_config = json.load(f)
    meta_config = models_config["meta"]
    ensemble_config = models_config["ensemble"]
    models_config = models_config["models"]
    
if "only_train_model" in meta_config:
    only_train_model = meta_config["only_train_model"]

only_train_model = "effnet-b5"

### Import preprocessing modules

In [ ]:
%run preprocessing.ipynb


### Load data

In [ ]:
class PreProcessCommonWrapper(object):
    def __init__(self, image_dim):
        self.image_dim = image_dim
        self.__name__ = "PreProcessCommonWrapper"
        self.__annotations__ = {}
    def __call__(self, t): # the function formerly known as "bar"
        return contrast_and_crop(t, self.image_dim)

def load_data(model_config):
    current_model_config = model_config
    print(current_model_config)
    base19_dir = os.path.join('../', 'input/aptos_image/')
    train19_dir = os.path.join('../', 'input/aptos_image/train_19/')
    base15_dir = os.path.join('../', 'input/aptos_image/')
    train15_dir = os.path.join('../', 'input/aptos_image/train_15/')
    
    df = pd.read_csv(os.path.join(base19_dir, 'labels/trainLabels19.csv'))
    
    df15 = pd.read_csv(os.path.join(base15_dir, 'labels/trainLabels15.csv'))
    
    # change id_code to accessible path and drops the id_code col
    df['path'] = df['file_name'].map(lambda x: os.path.join(train19_dir,f'{x}.jpg'))
    df = df.drop(columns=['file_name'])
    df15['path'] = df15['file_name'].map(lambda x: os.path.join(train15_dir,f'{x}.jpg'))
    df15 = df15.drop(columns=['file_name'])
    
    # add extras to training set
    df = pd.concat([df,df15], ignore_index=True)
    
    src = ImageList.from_df(df=df, path = './', cols='path') \
                   .split_by_rand_pct(seed=42) \
                   .label_from_df(cols='diagnosis', label_cls=FloatList)  # although labels are in integer form, they are intepreted as Float for training purposes
            
    transformations = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.3,max_lighting=0.1,p_lighting=0.5)
    
    # custom pre-processing (contrast and crop)
    pre_process_common_wrapper = PreProcessCommonWrapper(model_config["image_dim"])
    pre_process_ccs = [TfmPixel(pre_process_common_wrapper)()]
    advprop = model_config["advprop"]
    if advprop:
        pre_process_ccs.append(TfmPixel(advprop_normalise)())
    # apply transformations to training set, but apply the pre_process to train and valid set
    tfms = [transformations[0] + pre_process_ccs, transformations[1] + pre_process_ccs]
    
    # transform data sets
    data = src.transform(tfms, size=model_config["image_dim"], resize_method=ResizeMethod.CROP,padding_mode='zeros',) \
              .databunch(bs=model_config["batch_size"], num_workers=1) \
              .normalize(imagenet_stats if not advprop else None) # default normalise with imagenet stats, prebuilt into fast.ai library    
    
    print("loaded data")
    return (df, data)

In [ ]:
# lets visualise what we have got
import warnings 
warnings.filterwarnings("ignore")
# df, data = load_data(models_config["effnet-b3"])
# data.show_batch(rows=3, figsize=(10,10))

### Helper functions in loading models

In [ ]:
def getModel(model_name, data, model_dir=None, advprop=False, **kwargs):
    from os.path import abspath
    if model_dir is not None:
        model_dir = abspath(model_dir)
    model = EfficientNet.from_pretrained(model_name, advprop=advprop)
    model._fc = nn.Linear(model._fc.in_features,data.c) # .c returns number of output cells, ._fc returns the module
    return model

def get_learner(model_name, data, model_dir="models/", advprop=False):
    return Learner(data, getModel(model_name, data, model_dir=model_dir, advprop=advprop), metrics = [quadratic_kappa]) \
           .mixup() \
           .to_fp16() 

# quadratic kappa score
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    y_hat = y_hat.cpu()
    y = y.cpu()
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device='cuda:0')

### Train main models routine

In [ ]:
valid_predictions = {}
valid_labels = []

def train_nets():
    for config_name in models_config:

        if only_train_model and only_train_model != config_name:
            print(f"---- TRAINING SKIPPED FOR {config_name} ---")
            continue # skip this model
            # only train one model
            
        print(f"---- TRAINING STARTING FOR {config_name} ---")
        config = models_config[config_name]
        df, data = load_data(config)
        learner = get_learner(config["pretrained_name"], data, model_dir=config["pretrained_path"], advprop=config["advprop"])
        lr = config["lr"]
        # learner.lr_find()
        # learner.recorder.plot()
        # break

        # try to get the previous epochs, if any
        path = config["pretrained_path"]
        onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
        start_epoch = None
        for fn in onlyfiles:
            temp = fn.replace('.pth', '').split('_')
            if temp[0] == config_name and temp[1] == 'e':
                if start_epoch is None or int(temp[2]) > start_epoch:
                    start_epoch = int(temp[2])
        if start_epoch is not None:
            start_epoch += 1
        
        # training starts here 
        learner.fit_one_cycle(
            config["epoch_n"], 
            lr,
            start_epoch=start_epoch,
            callbacks=[
                SaveModelCallback(learner, every='epoch', monitor='valid_loss', name=f'{config_name}_e'),
                # SaveModelCallback(learner, every='improvement', monitor='valid_loss', name=f'{config_name}_best')
            ]
        )
        # learner.save(f'{config_name}_{int(datetime.now().timestamp())}')
        
        # perform prediction on 
        valid_predictions[config_name], valid_labels = learner.get_preds(DatasetType.Valid)
        valid_predictions[config_name] = valid_predictions[config_name].flatten().tolist()
        del learner
        # gc.collect()


### Ensemble layer

In [ ]:
def train_ensemble_average():
    valid_predictions["diagnosis"] = valid_labels
    valid_df = pd.DataFrame(valid_predictions)
    procs = [Normalize]
    ensemble_bunch = TabularList.from_df(df=valid_df, cont_names=model_names, procs=procs) \
                                .split_by_rand_pct() \
                                .label_from_df(cols='diagnosis') \
                                .databunch()
    
    learner = tabular_learner(ensemble_bunch, layers=[100, 50], ps=[0.4,0.2], metrics=[quadratic_kappa]) 
    # learner.lr_find()
    # learner.recorder.plot()
    lr = ensemble_config["lr"]
    epoch_n = ensemble_config["epoch_n"]
    learner.fit_one_cycle(
        epoch_n, 
        lr, 
        callbacks=[SaveModelCallback(learner, every='improvement', monitor='valid_loss', name="ensemble")]
    )


### Entrypoint

In [ ]:
if __name__ == '__main__':
    import warnings 
    warnings.filterwarnings("ignore")
    train_nets()
    train_ensemble_average()

In [ ]:
%ls ./../input/aptos_image/train_15/